<a href="https://colab.research.google.com/github/desstaw/Practical_ML_SS21/blob/main/Assignment8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Exercise Sheet 8**
**Group members**

Kristina Matz - 7020180

Despina Tawadros - 7933844

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn import datasets

In [ ]:
#Load Iris dataset
iris = datasets.load_iris()

#Store the data
X = iris.data

#Show features
print(*iris.feature_names)

sepal length (cm) sepal width (cm) petal length (cm) petal width (cm)


#Exercise 1

1)

In [ ]:
class LinearRegression:
  def __init__(self, lr):
    self.lr = lr
    

  def fit(self, X,Y):
    """
    Linear Regression: Y = mX * c
    """
    self.w = 0
    self.b = 0

    num_samples = len(X)
    epochs = 100000

    # Gradient descent
    for i in range(epochs):
      y_pred = self.predict(X)

      error = y_pred - Y


      #gradients
      dw = (1 / num_samples) * np.dot(X.T,error)
      db = (1 / num_samples) * np.sum(error)

      
      #Calculate and update the new parameters
      self.w -= self.lr * dw
      self.b -= self.lr * db

  def predict(self, X):
    # Predict targets using learned parameters
    return np.dot(X, self.w) + self.b

In [ ]:
# Set number of training samples
num_train = 100

# Get the training set
X_train = X[:num_train, :2]
Y_train = X[:num_train, 2:]

# Get test set
X_test = X[num_train:, :2]
Y_test = X[num_train:, 2:]

# Create regression model and fit to training set
learning_rate = 0.001
model = LinearRegression(learning_rate)
model.fit(X_train, Y_train)

# Predict target features in the test set
Y_test_pred = model.predict(X_test)

"""
Mean Squared Error
    - Find the difference between the actual y and predicted y value(y = mx + c), for a given x.
    - Square this difference.
    - Find the mean of the squares for every value in X.
"""
difference_array = np.subtract(Y_test, Y_test_pred)
squared_array = np.square(difference_array)

mse = squared_array.mean()

print("Mean Squared Error on test set:", mse)

Mean Squared Error on test set: 0.6788922425495524


2)

In [ ]:
class LinearRegression2:
  def __init__(self, lr, regularization=0):
    self.lr = lr
    self.reg = regularization
    

  def fit(self, X,Y):
    """
    Linear Regression: Y = mX * c
    """
    self.w = 0
    self.b = 0

    num_samples = len(X)
    epochs = 1000
    
    if self.reg != 0:
      # Gradient descent using L2 regularization
      for i in range(epochs):
        y_pred = self.predict(X)

        error = (y_pred - Y) + self.reg * np.sum(self.w ** 2)


        #gradients
        dw = (1 / num_samples) * np.dot(X.T,error)
        db = (1 / num_samples) * np.sum(error)

      
        #Calculate and update the new parameters
        self.w -= self.lr * dw
        self.b -= self.lr * db

    else:
      # Gradient descent
      for i in range(epochs):
        y_pred = self.predict(X)

        error = y_pred - Y


        #gradients
        dw = (1 / num_samples) * np.dot(X.T,error)
        db = (1 / num_samples) * np.sum(error)

      
        #Calculate and update the new parameters
        self.w -= self.lr * dw
        self.b -= self.lr * db


  def predict(self, X):
    # Predict targets using learned parameters
    return np.dot(X, self.w) + self.b

In [ ]:
# Cross validation

# Set number of folds
num_folds = 5

# Define values for regularization
choices_C = [0.001, 0.01, 0.1, 1, 10]

# Create matrix to store validation
results = np.empty(len(choices_C))

# Partition training set
X_train_folds = np.array_split(X_train, num_folds)
y_train_folds = np.array_split(Y_train, num_folds)

for i, C in enumerate(choices_C):
  # Create array to store intermediate results
    accuracies = np.empty(num_folds)

    # Cross validate hyperparameters setting
    for k in range(num_folds):

      # Use current fold for validation
      X_val = X_train_folds[k]
      y_val = y_train_folds[k]

      # Use remaining folds as testing set
      X_train_concat = np.concatenate(X_train_folds[:k] + X_train_folds[k + 1:])
      y_train_concat = np.concatenate(y_train_folds[:k] + y_train_folds[k + 1:])

      # Create support vector classifier
      model_2 = LinearRegression2(0.0001, C)

      # Fit the model to training data
      model_2.fit(X_train_concat, y_train_concat)

      # Compute accuracy
      pred = model_2.predict(X_val)
      difference_array = np.subtract(y_val, pred)
      squared_array = np.square(difference_array)

      mse = squared_array.mean()
      accuracies[k] = mse

# Get indices of best values
i = np.argmin(results)

best_C = choices_C[i] # best regularization parameter

print("Mean Squared Error with regularization parameter", best_C, ":", accuracies[i])


Mean Squared Error with regularization parameter 0.1 : 1.9537665392103007


3)

In [ ]:
class LinearRegression3:
  #1. here momentum
  def __init__(self, lr, momentum):
    self.lr = lr
    self.momentum = momentum
    

  def fit(self, X,Y):
    """
    Linear Regression: Y = mX * c
    """
    self.w = 0
    self.b = 0

    num_samples = len(X)
    epochs = 2

    # Gradient descent
    for i in range(epochs):
      y_pred = self.predict(X)

      error = y_pred - Y
      
      change_dw = 0.0
      change_db = 0.0

      # Gradients
      dw = (1 / num_samples) * np.dot(X.T,error)
      db = (1 / num_samples) * np.sum(error)      

      
      # Calculate and update the new parameters
      self.w -= self.lr * dw
      self.b -= self.lr * db

      new_change_dw = (i * dw) + (momentum * change_dw)
      new_change_db = (i * db) + (momentum * change_db)

      self.w -= (self.lr * dw) - new_change_dw
      self.b -= (self.lr * db) - new_change_db

      # Save the change
      change_dw = new_change_dw
      change_db = new_change_db

  def predict(self, X):
    # Predict targets using learned parameters
    return np.dot(X, self.w) + self.b

In [ ]:
# Set number of training samples
num_train = 100

# Get the training set
X_train = X[:num_train, :2]
Y_train = X[:num_train, 2:]

# Get test set
X_test = X[num_train:, :2]
Y_test = X[num_train:, 2:]

# Create regression model and fit to training set
learning_rate = 0.001
momentum = 0.9
model = LinearRegression3(learning_rate, momentum)
model.fit(X_train, Y_train)

# Predict target features in the test set
Y_test_pred = model.predict(X_test)

"""
Mean Squared Error
    - Find the difference between the actual y and predicted y value(y = mx + c), for a given x.
    - Square this difference.
    - Find the mean of the squares for every value in X.
"""
difference_array = np.subtract(Y_test, Y_test_pred)
squared_array = np.square(difference_array)

mse = squared_array.mean()

print("Mean Squared Error on test set:", mse)




Mean Squared Error on test set: 10239.450947111496
